# CUHK [STAT3009](https://www.bendai.org/STAT3009/) Notebook1: Baseline methods for Recommender Systems

## Software prepare
- `Code Editor`: VS Code; Sublime; or Atom

- `Terminal`: Iterm2 in Mac; Deepin terminal in Linux

## Creating virtual environments
- If you have multiple versions of Python on your system, you can select a specific Python version by running python3 or whichever version you want.

- To create a virtual environment, decide upon a directory where you want to place it, and run the venv module as a script with the directory path:

- How to create and activate a virtual environment, see Section 12.2 in the [Document]((https://docs.python.org/3/tutorial/venv.html)).

- Install packages via `pip`, see `Installing packages` section in the [Document]((https://docs.python.org/3/tutorial/venv.html)).


## Kaggle competition about recommender systems: user and item can be extended to more general cases.
- [Elo Merchant Category Recommendation](https://www.kaggle.com/c/elo-merchant-category-recommendation/data?select=Data+Dictionary.xlsx): `merchant_id` and `card_id`.

- [WSDM - KKBox's Music Recommendation Challenge](https://www.kaggle.com/c/kkbox-music-recommendation-challenge/data): `user` and `music`.

- [Event Recommendation Engine Challenge](https://www.kaggle.com/c/event-recommendation-engine-challenge/overview/evaluation): `user` and `event`.

## Load dataset into Python
- Dowload [Netflix Prize Data](https://www.kaggle.com/netflix-inc/netflix-prize-data). (For illustration, we only take the first subset.)

- Load data into Python.

- Re-orginize the data structure as a standard form.

- For testing set, we hide the real ratings.

- We only take the first subset for illustration.

In [2]:
import numpy as np
import pandas as pd

dtrain = pd.read_csv('./dataset/train.csv')
dtrain.sample(5)

,movie_id,user_id,rating,date
15340,408,1868,4,2005-06-12
204055,1746,7057,4,2004-06-14
181828,4231,2292,5,2003-10-06
74849,1111,387,3,2004-01-23
229588,2261,7972,5,2005-02-19


In [3]:
dtest = pd.read_csv('./dataset/test.csv')
## save real ratings for test set for evaluation.
test_ratings = np.array(dtest['rating'])
## remove the ratings in the test set to simulate prediction
dtest = dtest.drop(columns='rating')
dtest.sample(5)

,movie_id,user_id,date
149882,4337,1894,2005-04-02
252986,1516,1731,2005-07-06
171732,79,8260,2003-05-19
159834,3552,4072,2005-06-27
152554,677,6868,2005-03-25


## Pre-process the data as a `np.array`

In [4]:
## convert string to user_id and item_id -> [user_id, item_id, rating]
# pre-process for training data
train_pair = dtrain[['user_id', 'movie_id']].values
train_ratings = dtrain['rating'].values
# pre-process for testing set
test_pair = dtest[['user_id', 'movie_id']].values

n_user, n_item = max(train_pair[:,0].max(), test_pair[:,0].max())+1, max(train_pair[:,1].max(), test_pair[:,1].max())+1

## Implement Baseline methods: global\_average, user\_average and item\_average (For your practice)
- Inpout: training set.

- Output: return predicted ratings for (user id, item id) user-item pairs in testing set.

- Goal: make prediction for testing set

In [5]:
pred = np.zeros(len(test_ratings))

In [7]:
## Global average
global_pred = pred.copy()
global_mean = train_ratings.mean()
global_pred = global_mean*np.ones(len(pred))
print(global_pred[:10])

[3.60783271 3.60783271 3.60783271 3.60783271 3.60783271 3.60783271
 3.60783271 3.60783271 3.60783271 3.60783271]


### user\_average
- Loop for all users
    - Find all records for this user in both training and testing sets.
    - Compute the average ratings for this user in the training set.
    - Predict the ratings for this users in the testing set.

In [8]:
## user average
UA_pred = pred.copy()
for u in range(n_user):
    # find the index for both train and test for user_id = u
    ind_test = np.where(test_pair[:,0] == u)[0]
    ind_train = np.where(train_pair[:,0] == u)[0]
    if len(ind_test) == 0:
        continue
    if len(ind_train) < 3:
        UA_pred[ind_test] = global_mean
    else:
        # predict as user average
        UA_pred[ind_test] = train_ratings[ind_train].mean()
print(UA_pred[:10])

[3.62790698 4.03875969 3.41935484 3.52795031 3.6875     3.48888889
 3.33550489 4.68421053 3.44217687 3.75471698]


## Evaluation: compute RMSE for baseline methods
- Input: (1) predicted testing ratings (2) real testing ratings

- Output: RMSE for the prediction

- Goal: evaluate the prediction performance for the method.

In [9]:
## RMSE for Global average
rmse_glb = np.sqrt(np.mean((global_pred - test_ratings)**2))
print('RMSE for GLB average: %.3f' %rmse_glb)

RMSE for GLB average: 1.084


In [10]:
## RMSE for user average
rmse_usr = np.sqrt(np.mean((UA_pred - test_ratings)**2))
print('RMSE for user average: %.3f' %rmse_usr)

RMSE for user average: 1.014


## Summarize `glb_average` and `user_average` methods as Python functions

### `glb_average`

- *Input*: 'train_ratings', 'test_pair'

- *Return*: Predicted ratings based on glb mean.

In [11]:
def glb_mean(train_ratings, test_pair):
    pred = train_ratings.mean() * np.ones(len(test_pair))
    return pred

### `user_average`

- *Input*: 'train_pair', 'train_ratings', 'test_pair'

- *Return*: Predicted ratings based on user mean.

In [12]:
def user_mean(train_pair, train_ratings, test_pair):
    n_user = train_pair[:,0].max()+1
    pred = np.zeros(len(test_pair))
    glb_mean_value = train_ratings.mean()
    for u in range(n_user):
        # find the index for both train and test for user_id = u
        ind_test = np.where(test_pair[:,0] == u)[0]
        ind_train = np.where(train_pair[:,0] == u)[0]
        if len(ind_test) == 0:
            continue
        if len(ind_train) < 3:
            pred[ind_test] = glb_mean_value
        else:
            # predict as user average
            pred[ind_test] = ratings[ind_train].mean()
    return pred

## Summarize `Evaluation` as a Python function

In [13]:
def rmse(true, pred):
    return np.sqrt(np.mean((pred - true)**2))